# 언어 모델을 이용한 응용 프로그램 작동 방식

## 언어 모델 호출이란

In [1]:
import os
from openai import OpenAI

In [2]:
client = OpenAI(
    # api_key=os.environ.get("OPENAI_API_KEY"),
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "iphone8의 출시일을 yyyy/mm/dd 형식으로 알려줘.",
        }
    ],
    model="gpt-3.5-turbo",
)

print(response.choices[0].message.content)

죄송하지만 iphone8은 이미 출시된 제품이 아니기 때문에 정확한 출시일을 알려드릴 수는 없습니다. 현재 애플사의 최신 제품은 iPhone 13 시리즈입니다.


## Language models을 사용해 gpt-3.5-turbo 호출하기

In [1]:
import os
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
chat = ChatOpenAI(model="gpt-3.5-turbo-0125", 
                  temperature=0)

In [3]:
result = chat.invoke([
    HumanMessage(content='안녕하세요')
])
print(result.content)

안녕하세요! 무엇을 도와드릴까요?


- AIMessage 를 사용해 언어 모델 응답 표현

In [4]:
result = chat.invoke([
    HumanMessage(content='계란찜 만드는 법 알려줘'),    
    AIMessage(content="{ChatModel의 답변인 계란찜 만드는 법}"),
    HumanMessage(content='영어로 번역해줘')
])
print(result.content)

How to make steamed eggs


- SystemMessage을 사용해 언어 모델의 성격과 설정 정의

In [5]:
result = chat.invoke([
    SystemMessage(content='당신은 친한 친구 입니다. 존댓말을 쓰지 말고 솔직하게 답해줘.'),    
    HumanMessage(content='안녕? 잘 지냈어?')
])
print(result.content)

응, 안녕! 너도 잘 지냈어? 무슨 일 있었어?


- 언어 모델 교체

In [7]:
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    # api_key=os.environ.get("ANTHROPIC_API_KEY"),
)
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0,
    system="Today is March 4, 2024.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What are 3 ways to cook apples?"
                }
            ]
        }
    ]
)
print(message.content)

[TextBlock(text='Here are three popular ways to cook apples:\n\n1. Baked Apples: Core the apples and fill the center with a mixture of butter, brown sugar, cinnamon, and nutmeg. Place the apples in a baking dish with a little water, and bake in the oven until they are tender and caramelized.\n\n2. Apple Sauce: Peel, core, and chop the apples. Place them in a pot with a little water, sugar, and cinnamon. Cook over medium heat until the apples are soft and mushy. Mash the apples with a fork or blend them in a food processor for a smoother consistency.\n\n3. Sautéed Apples: Peel, core, and slice the apples. Melt butter in a skillet over medium heat, and add the apple slices. Sprinkle with sugar and cinnamon, and cook until the apples are tender and lightly caramelized, stirring occasionally.\n\nThese are just a few examples, as there are many other ways to cook apples, such as in pies, tarts, cobblers, and crisps.', type='text')]


In [8]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

In [9]:
chat = ChatAnthropic(model_name="claude-3-opus-20240229", 
                  temperature=0)
                  # anthropic_api_key=os.environ.get("ANTHROPIC_API_KEY"))

In [10]:
system = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat

chain.invoke(
    {
        "input_language": "English",
        "output_language": "Korean",
        "text": "I love Python",
    }
)

AIMessage(content='저는 파이썬을 사랑합니다.', additional_kwargs={}, response_metadata={'id': 'msg_015kvdYRKybG8vTn4aC3ZEKR', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 22, 'output_tokens': 20}}, id='run-88973b30-4027-4e99-a1ad-29a0f42534dd-0', usage_metadata={'input_tokens': 22, 'output_tokens': 20, 'total_tokens': 42})

## PromptTemplate 변수를 프롬프트에 전개 하기

In [12]:
from langchain import PromptTemplate

In [13]:
prompt = PromptTemplate(
    template = "{product}는 어느 회사에서 개발한 제품인가?",
    input_variables=[
        "product"
    ]
)

In [14]:
print(prompt.format(product="아이폰"))

아이폰는 어느 회사에서 개발한 제품인가?


## PromptTemplate에서 제공하는 다른 기능들

## Language models와 PromptTemplate의 결합

In [15]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

In [16]:
chat = ChatOpenAI(
    model="gpt-3.5-turbo-0125", 
    temperature=0, 
    api_key=os.environ.get("OPENAI_API_KEY")
)

/tmp/ipykernel_195934/176158663.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  chat = ChatOpenAI(


In [17]:
prompt = PromptTemplate(
    template = "{product}는 어느 회사에서 개발한 제품인가요?",
    input_variables=[
        "product"
    ]
)

In [18]:
result = chat.invoke(
    [
        HumanMessage(content=prompt.format(product="아이폰"))
    ]
)

In [19]:
print(result.content)

아이폰은 미국의 애플(Apple)사에서 개발한 제품입니다.


- PromptTemplate 초기화 방법의 종류

In [22]:
prompt = PromptTemplate.from_template("{product}는 어느 회사에서 개발한 제품인가요?")

## 목록 형식으로 결과 받기

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.schema import HumanMessage

In [24]:
output_parser = CommaSeparatedListOutputParser()
chat = ChatOpenAI(
    model="gpt-3.5-turbo-0125", 
    temperature=0, 
    api_key=os.environ.get("OPENAI_API_KEY")
)

In [25]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [26]:
result = chat.invoke(
    [
        HumanMessage(content="애플이 개발한 대표적인 제품 3개를 알려줘"),
        HumanMessage(content=output_parser.get_format_instructions())
    ]
)

In [27]:
result.content

'아이폰, 아이패드, 맥북'

In [28]:
output = output_parser.parse(result.content)

In [29]:
output

['아이폰', '아이패드', '맥북']

In [30]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

# Language models - 사용하기 쉬운 모델

In [52]:
from langchain.llms import OpenAI

llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    api_key=os.environ.get("OPENAI_API_KEY")
)

In [53]:
result = llm.invoke(
    "맛있는 라면을",
    stop="."
)

In [54]:
print(result)

 먹음");
+    }
+
+    @Override
+    protected void goHome() {
+        System


## Language models의 편리한 기능

### 캐싱

In [31]:
import time
import langchain
from langchain.cache import InMemoryCache
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

langchain.llm_cache = InMemoryCache()

chat = ChatOpenAI(
    model="gpt-3.5-turbo-0125", 
    temperature=0, 
    api_key=os.environ.get("OPENAI_API_KEY")
)

In [32]:
start = time.time()

result = chat.invoke(
    [
        HumanMessage(content="안녕하세요!")
    ]
)

end = time.time()
print(result.content)
print(f'실행 시간 : {end - start}초')

안녕하세요! 무엇을 도와드릴까요?
실행 시간 : 0.7028019428253174초


In [33]:
start = time.time()

result = chat.invoke(
    [
        HumanMessage(content="안녕하세요!")
    ]
)

end = time.time()
print(result.content)
print(f'실행 시간 : {end - start}초')

안녕하세요! 무엇을 도와드릴까요?
실행 시간 : 0.0008947849273681641초


### 결과를 순차적으로 표시

In [34]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

In [35]:
chat = ChatOpenAI(
    streaming=True, 
    model="gpt-3.5-turbo-0125", 
    temperature=0, 
    api_key=os.environ.get("OPENAI_API_KEY"),
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

In [36]:
resp = chat.invoke([
    HumanMessage(content="맛있는 스테이크 굽는 법을 알려주세요.")
])

맛있는 스테이크를 굽는 법은 다음과 같습니다:

1. 스테이크를 냉장고에서 꺼내어 실온에 30분 정도 방치하여 실온에 맞춰줍니다.
2. 팬이나 그릴을 중불로 예열합니다.
3. 스테이크에 소금과 후추를 골고루 뿌려줍니다.
4. 팬이나 그릴에 식용유를 두르고 스테이크를 올려줍니다.
5. 한쪽 면을 3분 정도 구워준 후 뒤집어 반대 면도 3분 정도 구워줍니다.
6. 스테이크의 두 측면을 각각 1분 정도씩 추가로 구워줍니다.
7. 스테이크를 꺼내어 알루미늄 호일에 싸서 5분 정도 쉬어두어 주세요.
8. 스테이크를 잘라서 그릇에 담고 즐겨주세요.

이렇게 하면 부드럽고 맛있는 스테이크를 즐길 수 있습니다. 추가로 스테이크의 익도는 개인의 취향에 맞게 조절해주시면 됩니다.

In [37]:
resp.content

'맛있는 스테이크를 굽는 법은 다음과 같습니다:\n\n1. 스테이크를 냉장고에서 꺼내어 실온에 30분 정도 방치하여 실온에 맞춰줍니다.\n2. 팬이나 그릴을 중불로 예열합니다.\n3. 스테이크에 소금과 후추를 골고루 뿌려줍니다.\n4. 팬이나 그릴에 식용유를 두르고 스테이크를 올려줍니다.\n5. 한쪽 면을 3분 정도 구워준 후 뒤집어 반대 면도 3분 정도 구워줍니다.\n6. 스테이크의 두 측면을 각각 1분 정도씩 추가로 구워줍니다.\n7. 스테이크를 꺼내어 알루미늄 호일에 싸서 5분 정도 쉬어두어 주세요.\n8. 스테이크를 잘라서 그릇에 담고 즐겨주세요.\n\n이렇게 하면 부드럽고 맛있는 스테이크를 즐길 수 있습니다. 추가로 스테이크의 익도는 개인의 취향에 맞게 조절해주시면 됩니다.'

In [ ]:
StreamingStdOutCallbackHandler()

# Templates - 프롬프트 구축의 효율성 향상

## 프롬프트 엔지니어링을 통한 결과 최적화

### 출력 예가 포함된 프롬프트 만들기

In [39]:
from langchain.llms import OpenAI
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

In [40]:
examples = [
    {
        "input":"충청도의 계룡산 전라도의 내장산 강원도의 설악산은 모두 국립 공원이다",
        "output":"충청도의 계룡산, 전라도의 내장산, 강원도의 설악산은 모두 국립 공원이다."
    }
]

In [41]:
prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="입력 : {input}\n출력 : {output}",
)

In [42]:
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt,
    prefix="아래 문장부호가 빠진 입력에 문장부호를 추가하세요. 추가할수 있는 문장 부호는 ',', '.' 입니다. 다른 문장부호는 추가하지 마세요.", 
    suffix="입력 : {input_string}\n출력",
    input_variables=["inpuit_string"],
)

In [43]:
llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    api_key=os.environ.get("OPENAI_API_KEY")
)

/tmp/ipykernel_195934/2125856350.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm = OpenAI(


In [45]:
formatted_prompt = few_shot_prompt.format(
    input_string="집을 보러 가면 그 집이 내가 원하는 조건에 맞는지 살기 편한지 망가진 곳은 없는지 확인 해야 한다."
)

In [46]:
result = llm.invoke(formatted_prompt)

In [47]:
print(formatted_prompt)

아래 문장부호가 빠진 입력에 문장부호를 추가하세요. 추가할수 있는 문장 부호는 ',', '.' 입니다. 다른 문장부호는 추가하지 마세요.

입력 : 충청도의 계룡산 전라도의 내장산 강원도의 설악산은 모두 국립 공원이다
출력 : 충청도의 계룡산, 전라도의 내장산, 강원도의 설악산은 모두 국립 공원이다.

입력 : 집을 보러 가면 그 집이 내가 원하는 조건에 맞는지 살기 편한지 망가진 곳은 없는지 확인 해야 한다.
출력


In [48]:
print(result)

 : 집을 보러 가면, 그 집이 내가 원하는 조건에 맞는지 살기 편한지 망가진 곳은 없는지 확인 해야 한다.


# Output parsers - 출력 구조화

## 결과를 날씨와 시간 형식으로 받아보기

In [87]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import DatetimeOutputParser
from langchain.schema import HumanMessage

output_parser = DatetimeOutputParser()

chat = ChatOpenAI(
    model="gpt-3.5-turbo-0125", 
    temperature=0, 
    api_key=os.environ.get("OPENAI_API_KEY")
)

prompt = PromptTemplate.from_template(
    "{product}의 출시일을 알려주세요."
)

result = chat.invoke(
    [
        HumanMessage(content=prompt.format(product='iPhone8')),
        HumanMessage(content=output_parser.get_format_instructions())
    ]
)

In [88]:
output = output_parser.parse(result.content)
print(output)

2022-09-22 00:00:00


In [89]:
output_parser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 11-12-04T05:14:15.751033Z, 1347-08-30T10:43:52.547134Z, 1139-08-04T05:43:31.533813Z\n\nReturn ONLY this string, no other words!"

## 출력 형식을 직접 정의하기

In [49]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

/home/restful3/anaconda3/envs/langchain/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [50]:
model = ChatOpenAI(
    model="gpt-3.5-turbo-0125", 
    temperature=0, 
    api_key=os.environ.get("OPENAI_API_KEY")
)

In [107]:
class Smartphone(BaseModel):
    release_date: str = Field(description="스마트폰 출시일")
    screen_inches: float = Field(description="스마트폰의 화면 크기(인치)")
    os_installed: str = Field(description="스마트폰에 설치된 OS")
    model_name: str = Field(description="스마트폰 모델명")

    @validator('screen_inches')
    def validate_screen_inches(cls, field):
        if field <=0:
            raise ValueError("Screen inches must be a positive number")
        return field

# And a query intented to prompt a language model to populate the data structure.
my_query = "안드로이드 스마트폰 1개를 꼽아주세요."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Smartphone)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

result = chain.invoke({"query": my_query})
result

Smartphone(release_date='2021-10-15', screen_inches=6.5, os_installed='Android 11', model_name='Samsung Galaxy S21')

In [114]:
print(f'모델명 : {result.model_name}')
print(f'화면크기 : {result.screen_inches}')
print(f'OS : {result.os_installed}')
print(f'스마트폰 출시일 : {result.release_date}')

모델명 : Samsung Galaxy S21
화면크기 : 6.5
OS : Android 11
스마트폰 출시일 : 2021-10-15


## 잘못된 결과가 반환될 때 수정을 지시할 수 있게 된다

In [15]:
from langchain.output_parsers import OutputFixingParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

In [16]:
model = ChatOpenAI(
    model="gpt-3.5-turbo-0125", 
    temperature=0, 
    api_key=os.environ.get("OPENAI_API_KEY")
)

In [17]:
class Smartphone(BaseModel):
    release_date: str = Field(description="스마트폰 출시일")
    screen_inches: float = Field(description="스마트폰의 화면 크기(인치)")
    os_installed: str = Field(description="스마트폰에 설치된 OS")
    model_name: str = Field(description="스마트폰 모델명")

    @validator('screen_inches')
    def validate_screen_inches(cls, field):
        if field <=0:
            raise ValueError("Screen inches must be a positive number")
        return field

In [18]:
parser = OutputFixingParser.from_llm(
    parser=PydanticOutputParser(pydantic_object=Smartphone),
    llm=model
)

In [19]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


In [20]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"release_date": {"title": "Release Date", "description": "\uc2a4\ub9c8\ud2b8\ud3f0 \ucd9c\uc2dc\uc77c", "type": "string"}, "screen_inches": {"title": "Screen Inches", "description": "\uc2a4\ub9c8\ud2b8\ud3f0\uc758 \ud654\uba74 \ud06c\uae30(\uc778\uce58)", "type": "number"}, "os_installed": {"title": "Os Installed", "description": "\uc2a4\ub9c8\ud2b8\ud3f0\uc5d0 \uc124\uce58\ub41c OS", "type": "string"}, "model_name": {"title": "Model Name", "description": "\uc2a4\ub9c8\ud2b8\ud3f0 \ubaa8\ub378\uba85", "type": "string"}}, "requi

In [21]:
chain = prompt | model | parser

my_query = "안드로이드 스마트폰 1개를 꼽아주세요"

result = chain.invoke({"query": my_query})
result

Smartphone(release_date='2021-10-15', screen_inches=6.5, os_installed='Android 11', model_name='Samsung Galaxy S21')

In [22]:
print(f'모델명 : {result.model_name}')
print(f'화면크기 : {result.screen_inches}')
print(f'OS : {result.os_installed}')
print(f'스마트폰 출시일 : {result.release_date}')

모델명 : Samsung Galaxy S21
화면크기 : 6.5
OS : Android 11
스마트폰 출시일 : 2021-10-15
